In [14]:
%%bigquery

with size_bucket as (
SELECT size , count(size) number_elements,
RANGE_BUCKET(size, [0, 1,10, 100000, 200000, 400000 ,800000 ,1600000, 3200000 ,6400000 ]) AS bucket,
FROM `ml-demo-384110.immo_us.immo_images_files` group by size
)
select bucket, sum(number_elements) number_elements, min(size) minimum, max(size) maximum, avg(size ) average 
from size_bucket group by size_bucket.bucket order by size_bucket.bucket





Downloading: 100%|██████████| 9/9 [00:01<00:00,  5.06rows/s]


,bucket,number_elements,minimum,maximum,average
0,1,502,0,0,0.000000e+00
1,2,39,9,9,9.000000e+00
2,3,876,19,99615,7.340155e+04
3,4,1308,100041,199844,1.502054e+05
4,5,2535,200115,399859,2.860284e+05
5,6,1060,400372,798344,5.484830e+05
6,7,300,800528,1584561,1.023588e+06
7,8,42,1616884,3031683,1.980700e+06
8,9,2,4041052,5015510,4.528281e+06


In [15]:
%%bigquery df 

WITH Ids AS (
  SELECT uri AS uri, size, metadata
  from  `ml-demo-384110.immo_us.immo_images_files` 
)
select uri, size, id from (
SELECT
  uri, size, 
  value as id
FROM Ids AS r  CROSS JOIN UNNEST(r.metadata) AS metadata 
) 
where 
size = 9
;



Downloading: 100%|██████████| 39/39 [00:02<00:00, 17.81rows/s]


In [ ]:
df

In [10]:
import glob
import random
import base64
import pandas as pd
import requests

from PIL import Image
from io import BytesIO
from IPython.display import HTML

import io
from io import BytesIO

from google.cloud import storage

#pd.set_option('display.max_colwidth', -1)

def get_thumbnail(i, size=150):
    i.thumbnail((size, size), Image.LANCZOS)
    return i

def image_base64(im):
    if isinstance(im, str):
        im = get_thumbnail(im)
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    return f'<img src="data:image/jpeg;base64,{image_base64(im)}">'


def download_blob(bucket_name, source_blob_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(source_blob_name)


    blob_bytes =    BytesIO(blob.download_as_bytes())
    img = Image.open(blob_bytes)
    
    return img
    

def download_images(url):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"}

    try:
        #response = requests.get(url, stream=False)
        response = requests.get(url, stream=False, headers=headers)    

        if response.status_code == 200:
            blob_bytes = BytesIO(response.content)
            img = Image.open(blob_bytes)
            return img
        print(response.status_code)

    except:
        print("Image download failed")        
        return None


    

def add_images_thumbnail_to_df(df, size = 150, bucket_name = "ml-demo-384110-bucket"):

    df['object_name'] = df.uri.map(
        lambda uri_name: uri_name.replace(f"gs://{bucket_name}/", ""))
    df['image'] = df.object_name.map(lambda f: get_thumbnail(download_blob(bucket_name, f), size))
    return df

def add_web_images_thumbnail_to_df(df, size = 150):

    df['web_image'] = df.web_url.map(lambda f: get_thumbnail(download_images(f), size))
    return df

In [ ]:
df_automl_landmarks_images = add_images_thumbnail_to_df(df)
# displaying PIL.Image objects embedded in dataframe
HTML(df_automl_landmarks_images[[ 'image', id ]].to_html(formatters={'image': image_formatter}, escape=False))

In [18]:
%%bigquery

with size_bucket as (
SELECT size , count(size) number_elements,
# RANGE_BUCKET(size, [0, 1,10, 100000, 200000, 400000 ,800000 ,1600000, 3200000 ,6400000 ]) AS bucket,
ML.BUCKETIZE(size, [0, 1,10, 100000, 200000, 400000 ,800000 ,1600000, 3200000 ,6400000 ]) AS bucket,
#ML.QUANTILE_BUCKETIZE(size, 4) OVER() AS bucket

FROM `ml-demo-384110.immo_us.immo_images_files`
group by size
)
select bucket, sum(number_elements) number_elements, min(size) minimum, max(size) maximum,


avg(size ) average from size_bucket
group by size_bucket.bucket
order by size_bucket.bucket

Downloading: 100%|██████████| 9/9 [00:02<00:00,  3.69rows/s]


,bucket,number_elements,minimum,maximum,average
0,bin_10,2,4041052,5015510,4.528281e+06
1,bin_2,502,0,0,0.000000e+00
2,bin_3,39,9,9,9.000000e+00
3,bin_4,876,19,99615,7.340155e+04
4,bin_5,1308,100041,199844,1.502054e+05
5,bin_6,2535,200115,399859,2.860284e+05
6,bin_7,1060,400372,798344,5.484830e+05
7,bin_8,300,800528,1584561,1.023588e+06
8,bin_9,42,1616884,3031683,1.980700e+06


In [19]:
%%bigquery df

with size_bucket as (
SELECT size , count(size) number_elements,
RANGE_BUCKET(size, [0, 1,10, 100000, 200000, 400000 ,800000 ,1600000, 3200000 ,6400000 ]) AS bucket,
FROM `ml-demo-384110.immo_us.immo_images_files` group by size
)
select bucket, uri, size
from size_bucket 
#where size_bucket.bucket  = 2






Executing query with job ID: 37e9c877-a13b-41d1-b836-62f808a63180
Query executing: 0.58s


ERROR:
 400 Unrecognized name: uri at [6:16]

Location: US
Job ID: 37e9c877-a13b-41d1-b836-62f808a63180



In [2]:
%load_ext google.cloud.bigquery


In [ ]:
%%bigquery

create or replace view  `ml-demo-384110.immo_us.image_cloudvision_results` as (

with results as (
      SELECT 
      files.metadata as metadata, ml_res.* 
      except(metadata)
 FROM `ml-demo-384110.immo_us.temp_image_results_test` as ml_res inner join `ml-demo-384110.immo_us.immo_images_files` as files on ml_res.uri = files.uri
)            
  select *
from (
SELECT
  r.*, 
  metadata.value as metadata_value,
  metadata.name as metadata_name
FROM results AS r  CROSS JOIN UNNEST(r.metadata) AS metadata 
)
)